In [81]:
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt
import numpy as np

# Membaca df_ori
df_ori = pd.read_csv("Data_USD/data_prophet.csv")
df_ori["Tanggal"] = pd.to_datetime(df_ori["Tanggal"])
df_ori = df_ori[["Tanggal", "Harga_Jual","Inflasi","Suku_Bunga","TheFed"]]


display(df_ori)

,Tanggal,Harga_Jual,Inflasi,Suku_Bunga,TheFed
0,2015-01-02,12536.0,6.96,7.75,0.25
1,2015-01-05,12652.0,6.96,7.75,0.25
2,2015-01-06,12721.0,6.96,7.75,0.25
3,2015-01-07,12796.0,6.96,7.75,0.25
4,2015-01-08,12795.0,6.96,7.75,0.25
...,...,...,...,...,...
2303,2024-05-31,16334.0,2.84,6.25,5.50
2304,2024-06-03,16332.0,2.84,5.50,5.50
2305,2024-06-04,16306.0,2.84,5.50,5.50
2306,2024-06-05,16301.0,2.84,5.50,5.50


In [82]:
# Variable berapa hari yang akan diprediksi
jumlah_prediksi = 100

from prophet import Prophet
# 1. Dataframe
df_prophet = df_ori[
    ["Tanggal", "Harga_Jual", "Inflasi", "Suku_Bunga", "TheFed"]
]
df_prophet = df_prophet.rename(
    columns={"Tanggal": "ds"}
)  # Ubah nama kolom Tanggal menjadi 'ds'

# 2. Function
models = {}
forecasts = {}
for column in df_prophet.columns[1:]:
    model = Prophet(
        interval_width=0.8
    )
    model.fit(
        df_prophet[["ds", column]].rename(columns={column: "y"})
    )  # y adalah semua kolom yang akan diprediksi
    models[column] = model

    # 3. Buat Dataframe Masa Depan sesuai permintaan
    future = model.make_future_dataframe(periods=jumlah_prediksi)

    # 4. Gunakan Model untuk Memprediksi
    forecast = model.predict(future)
    forecast["yhat"] = forecast["yhat"].rolling(window=6).mean()
    forecasts[column] = forecast.tail(jumlah_prediksi)[["ds", "yhat"]]

# 5. Simpan hasil prediksi untuk setiap fitur ke dalam variabel tertentu
predicted_values = {}
for column, forecast in forecasts.items():
    predicted_values[column] = forecast["yhat"].values

predicted_dates = forecast["ds"].dt.strftime("%d/%m/%Y").values

# Ambil tanggal terakhir dalam data
last_date = df_prophet["ds"].max()

# 6. Buat dataframe dari hasil prediksi
predicted_df_multivariate = pd.DataFrame(
    {
        "Tanggal": pd.date_range(
            start=last_date + pd.Timedelta(days=1), periods=jumlah_prediksi
        ),
        "Inflasi": predicted_values["Inflasi"],
        "Suku_Bunga": predicted_values["Suku_Bunga"],
        "TheFed": predicted_values["TheFed"],
    }
)

# 7. Gabungkan dataframe hasil prediksi dengan dataframe asli
df = pd.concat([df_ori, predicted_df_multivariate], ignore_index=True)

15:54:26 - cmdstanpy - INFO - Chain [1] start processing
15:54:26 - cmdstanpy - INFO - Chain [1] done processing
15:54:28 - cmdstanpy - INFO - Chain [1] start processing
15:54:28 - cmdstanpy - INFO - Chain [1] done processing
15:54:29 - cmdstanpy - INFO - Chain [1] start processing
15:54:30 - cmdstanpy - INFO - Chain [1] done processing
15:54:30 - cmdstanpy - INFO - Chain [1] start processing
15:54:31 - cmdstanpy - INFO - Chain [1] done processing


In [83]:
# Prepare dataframe with additional regressors
df_with_regressor = df[
    ["Tanggal", "Harga_Jual","Inflasi","Suku_Bunga", "TheFed"]
]
df_with_regressor = df_with_regressor.rename(
    columns={"Tanggal": "ds", "Harga_Jual": "y"}
)

# Train Prophet model with additional regressors
model_with_regressor = Prophet(
    interval_width=0.7,
)

model_with_regressor.add_regressor("Inflasi", standardize=False)
model_with_regressor.add_regressor("Suku_Bunga", standardize=False)
model_with_regressor.add_regressor("TheFed", standardize=False)
model_with_regressor.fit(df_with_regressor)

15:54:32 - cmdstanpy - INFO - Chain [1] start processing
15:54:35 - cmdstanpy - INFO - Chain [1] done processing


In [84]:
multivariate_predicted = df[
    ["Inflasi", "Suku_Bunga", "TheFed"]
]  # .tail(2265)

# Prepare future dataframe for predictions Regressor
future_with_regressor = model_with_regressor.make_future_dataframe(
    periods=jumlah_prediksi, freq="D"
)

# Regressor
future_with_regressor["Inflasi"] = multivariate_predicted["Inflasi"].values
future_with_regressor["Suku_Bunga"] = multivariate_predicted["Suku_Bunga"].values
future_with_regressor["TheFed"] = multivariate_predicted["TheFed"].values

# Make predictions
forecast_with_regressor = model_with_regressor.predict(future_with_regressor)
forecast_with_regressor["yhat"] = (
    forecast_with_regressor["yhat"].rolling(window=7).mean()
)
forecast_with_regressor["yhat_lower"] = (
    forecast_with_regressor["yhat_lower"].rolling(window=7).mean()
)
forecast_with_regressor["yhat_upper"] = (
    forecast_with_regressor["yhat_upper"].rolling(window=7).mean()
)

In [85]:
from prophet.plot import plot_plotly, plot_components_plotly

plot_plotly(model_with_regressor, forecast_with_regressor)

In [86]:
import pandas as pd

# df dari df_ori
df_ori_1 = df_ori[(df_ori["Tanggal"] > "2015-01-30") & (df_ori["Tanggal"] <= "2024-05-20")]

df_eval_with_regressor = forecast_with_regressor[
    (forecast_with_regressor["ds"] > "2015-01-30")
    & (forecast_with_regressor["ds"] <= "2024-05-20")
]

#20 may

# Tampilkan hasil filter
print(df_ori_1[["Tanggal", "Harga_Jual"]].tail(5))
print("====================================")
print(df_eval_with_regressor[["ds", "yhat", "yhat_upper", "yhat_lower"]].tail(5))

        Tanggal  Harga_Jual
2292 2024-05-14     16165.0
2293 2024-05-15     16161.0
2294 2024-05-16     16134.0
2295 2024-05-17     16105.0
2296 2024-05-20     16174.0
             ds          yhat    yhat_upper    yhat_lower
2292 2024-05-14  16217.178975  16474.829771  15960.030165
2293 2024-05-15  16227.017028  16485.192913  15967.921770
2294 2024-05-16  16236.305218  16498.604128  15979.081934
2295 2024-05-17  16244.421189  16505.400967  15987.393650
2296 2024-05-20  16251.185310  16511.762032  15990.175031


In [87]:
display(df_ori_1)

,Tanggal,Harga_Jual,Inflasi,Suku_Bunga,TheFed
21,2015-02-02,12764.0,6.29,7.50,0.25
22,2015-02-03,12706.0,6.29,7.50,0.25
23,2015-02-04,12672.0,6.29,7.50,0.25
24,2015-02-05,12716.0,6.29,7.50,0.25
25,2015-02-06,12676.0,6.29,7.50,0.25
...,...,...,...,...,...
2292,2024-05-14,16165.0,3.00,6.25,5.50
2293,2024-05-15,16161.0,3.00,6.25,5.50
2294,2024-05-16,16134.0,3.00,6.25,5.50
2295,2024-05-17,16105.0,3.00,6.25,5.50


In [88]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Menghitung MSE
mse_with_regressor = mean_squared_error(
    df_ori_1["Harga_Jual"], df_eval_with_regressor["yhat"]
)

# Menghitung MSA
msa_with_regressor = mean_absolute_error(
    df_ori_1["Harga_Jual"], df_eval_with_regressor["yhat"]
)

# Menghitung RMSE
rmse_with_regressor = mean_squared_error(
    df_ori_1["Harga_Jual"], df_eval_with_regressor["yhat"], squared=False
)

print("====================================")
print("MSE With Regressor:", mse_with_regressor)
print("MSA With Regressor:", msa_with_regressor)
print("RMSE With Regressor:", rmse_with_regressor)


# Menghitung MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape_with_regressor = mean_absolute_percentage_error(
    df_ori_1["Harga_Jual"], df_eval_with_regressor["yhat"]
)

print("====================================")
print("MAPE With Regressor:", mape_with_regressor)

MSE With Regressor: 64484.51312332589
MSA With Regressor: 178.63275726730467
RMSE With Regressor: 253.93801039491092
MAPE With Regressor: 1.2397600091632413


In [89]:
# kebutuhan export
start_date = "2024-01-01"
end_date = "2024-05-27"


export_with_regressor = forecast_with_regressor[
    (forecast_with_regressor["ds"] >= start_date)
    & (forecast_with_regressor["ds"] <= end_date)
]

export_with_regressor = export_with_regressor[
    ["ds", "yhat", "yhat_upper", "yhat_lower"]
]

def format_cleaning(value):
    value_str = str(value)
    if len(value_str) > 5:
        return value_str[:5]
    elif len(value_str) < 5:
        return value_str.ljust(5, "0")
    else:
        return value_str

export_with_regressor["yhat"] = export_with_regressor["yhat"].apply(format_cleaning)
export_with_regressor["yhat_upper"] = export_with_regressor["yhat_upper"].apply(
    format_cleaning
)
export_with_regressor["yhat_lower"] = export_with_regressor["yhat_lower"].apply(
    format_cleaning
)

import os

# Tentukan path file dan folder
file_path = "Hasil Prediksi USD Report CSV/Hasil_With_Regressor.csv"
folder_path = os.path.dirname(file_path)

# Cek apakah folder ada, jika tidak buat folder tersebut
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Simpan DataFrame ke file CSV
export_with_regressor.to_csv(file_path, index=False)

# export_with_regressor.to_csv(
#     "Hasil Prediksi USD Report CSV/Hasil_With_Regressor.csv", index=False
# )

print("=================With Regressor==================")
print(export_with_regressor)

=================With Regressor==================
             ds   yhat yhat_upper yhat_lower
2212 2024-01-02  15763      16027      15514
2213 2024-01-03  15755      16019      15505
2214 2024-01-04  15746      16008      15492
2215 2024-01-05  15736      15998      15483
2216 2024-01-08  15724      15985      15471
...         ...    ...        ...        ...
2295 2024-05-17  16244      16505      15987
2296 2024-05-20  16251      16511      15990
2297 2024-05-21  16251      16509      15990
2298 2024-05-22  16249      16507      15992
2299 2024-05-27  16243      16505      15991

[88 rows x 4 columns]
